In [1]:
%matplotlib inline

In [2]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

In [3]:
con=sqlite3.connect('./AmazonReviewDataset/database.sqlite')

In [4]:
data=pd.read_sql_query('''select * from Reviews where Score!=3''',con)

In [5]:
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
525809,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
525810,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
525811,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
525812,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [6]:
def partition(x):
    if x<3:
        return 'Negative'
    return 'Positive'

In [7]:
s=data['Score']
pn=s.map(partition)
data['Score']=pn

In [8]:
data['Score']

0         Positive
1         Negative
2         Positive
3         Negative
4         Positive
            ...   
525809    Positive
525810    Negative
525811    Positive
525812    Positive
525813    Positive
Name: Score, Length: 525814, dtype: object

DATA CLEANING

In [9]:
sd=data.sort_values('ProductId',axis=0,ascending=True)

In [10]:
sd

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
...,...,...,...,...,...,...,...,...,...,...
176791,191721,B009UOFTUI,AJVB004EB0MVK,D. Christofferson,0,0,Negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
1362,1478,B009UOFU20,AJVB004EB0MVK,D. Christofferson,0,0,Negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
303285,328482,B009UUS05I,ARL20DSHGVM1Y,Jamie,0,0,Positive,1331856000,Perfect,The basket was the perfect sympathy gift when ...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [11]:
final=sd.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)

In [12]:
final

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
...,...,...,...,...,...,...,...,...,...,...
178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,Positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m..."
173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,Positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...
204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,Positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [13]:
final=final[final['HelpfulnessNumerator']<=final['HelpfulnessDenominator']]

In [14]:
final

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
...,...,...,...,...,...,...,...,...,...,...
178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,Positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m..."
173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,Positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...
204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,Positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,Positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [15]:
final['Score'].value_counts()

Score
Positive    307061
Negative     57110
Name: count, dtype: int64

In [52]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Aashraey
[nltk_data]     Vaid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop=set(stopwords.words('english'))

In [19]:
print(stop)

{"hasn't", "aren't", 'should', 'why', 'below', 'here', 'haven', 'over', "weren't", 'under', 'when', "he's", 'or', "they've", 'above', 'did', 'up', 're', 'o', 'very', 't', "we'll", "you'll", 's', 'of', 'then', 'more', 'no', 'doing', "should've", 'there', 'that', 'before', "haven't", 'into', 'each', 'same', "wasn't", "i'm", "we'd", 'some', 'himself', 'such', 'their', 'to', "he'll", 'needn', "needn't", 'yours', 'down', 'hadn', 'itself', 'where', 'his', 'because', 'ma', 'your', 'hers', 'll', 'off', 'this', 'after', 'ain', "you're", 'in', "mightn't", 'been', "he'd", 'myself', "she'll", 'theirs', 'but', 'too', 'him', 'and', 'at', 'so', 'ourselves', 'having', 'what', 'most', 'once', 'a', 'was', "we're", 'from', 'd', 'were', 'her', "couldn't", 'do', 'aren', "she's", 'wasn', "i'll", 'didn', "don't", 'through', 'again', "i've", 'now', 'these', 'have', 'not', "mustn't", 'out', 'she', "doesn't", 'he', 'our', 'mustn', 'for', "she'd", "didn't", "they'll", "they'd", 'mightn', 'are', 'i', 'my', 'by', 

In [21]:
count_vect=CountVectorizer()
final_count=count_vect.fit_transform(final['Text'].values)

In [22]:
type(final_count)

scipy.sparse._csr.csr_matrix

In [23]:
final_count.get_shape()

(364171, 115281)

In [25]:
i=0
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break
    i+=1

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [26]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [27]:
stop=set(stopwords.words('english'))                         #set of stopwords
sno=nltk.stem.SnowballStemmer('english')                     #initialize snowball stemmer

In [28]:
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext

In [29]:
def cleanpunc(sentence):
    cleaned=re.sub(r'[2|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

In [30]:
print(stop)

{"hasn't", "aren't", 'should', 'why', 'below', 'here', 'haven', 'over', "weren't", 'under', 'when', "he's", 'or', "they've", 'above', 'did', 'up', 're', 'o', 'very', 't', "we'll", "you'll", 's', 'of', 'then', 'more', 'no', 'doing', "should've", 'there', 'that', 'before', "haven't", 'into', 'each', 'same', "wasn't", "i'm", "we'd", 'some', 'himself', 'such', 'their', 'to', "he'll", 'needn', "needn't", 'yours', 'down', 'hadn', 'itself', 'where', 'his', 'because', 'ma', 'your', 'hers', 'll', 'off', 'this', 'after', 'ain', "you're", 'in', "mightn't", 'been', "he'd", 'myself', "she'll", 'theirs', 'but', 'too', 'him', 'and', 'at', 'so', 'ourselves', 'having', 'what', 'most', 'once', 'a', 'was', "we're", 'from', 'd', 'were', 'her', "couldn't", 'do', 'aren', "she's", 'wasn', "i'll", 'didn', "don't", 'through', 'again', "i've", 'now', 'these', 'have', 'not', "mustn't", 'out', 'she', "doesn't", 'he', 'our', 'mustn', 'for', "she'd", "didn't", "they'll", "they'd", 'mightn', 'are', 'i', 'my', 'by', 

In [31]:
print(sno.stem('tasty'))

tasti


In [44]:
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''

for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) and (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values[i]=='Positive'):
                        all_positive_words.append(s)
                    if(final['Score'].values[i]=='Negative'):
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    str1=b" ".join(filtered_sentence)
    final_string.append(str1)
    i+=1 

In [45]:
final.loc[:, 'cleanedText'] = final_string

conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')

364171

In [46]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most common positive words are: ",freq_dist_positive.most_common(20))
print("Most common negative words are: ",freq_dist_negative.most_common(20))

Most common positive words are:  [(b'like', 139214), (b'tast', 128885), (b'good', 112623), (b'flavor', 109505), (b'love', 107249), (b'use', 103862), (b'great', 103846), (b'one', 96671), (b'product', 90900), (b'tri', 86774), (b'tea', 83750), (b'coffe', 78687), (b'make', 75101), (b'get', 72091), (b'food', 64682), (b'would', 55566), (b'time', 55211), (b'buy', 54185), (b'realli', 52619), (b'eat', 51968)]
Most common negative words are:  [(b'tast', 34499), (b'like', 32308), (b'product', 28091), (b'one', 20529), (b'flavor', 19499), (b'would', 17970), (b'tri', 17745), (b'use', 15290), (b'good', 15011), (b'coffe', 14668), (b'get', 13774), (b'buy', 13746), (b'order', 12857), (b'food', 12697), (b'dont', 11876), (b'tea', 11624), (b'even', 11085), (b'box', 10791), (b'amazon', 10023), (b'make', 9837)]


In [47]:
count_vect=CountVectorizer(ngram_range=(1,2))
final_bigram_counts=count_vect.fit_transform(final['Text'].values)

final_bigram_counts.get_shape()

(364171, 2910192)

In [48]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)

In [49]:
final_tf_idf.get_shape()

(364171, 2910192)

In [54]:
features=tf_idf_vect.get_feature_names_out()
len(features)

2910192

In [55]:
features[100000:100000]

array([], dtype=object)

In [56]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [59]:
def top_tfidf_feats(row,features,top_n=25):
    '''Get top 10 tf-idf values in rows and return them with their tfidf value'''
    topn_ids=np.argsort(row)[::-1][:top_n]
    top_feats=[(features[i],row[i]) for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['feature','tfidf']
    return df

top_tfidf=top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [60]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074
